In [1]:
from datetime import datetime
from datetime import timedelta

from pymongo import MongoClient
from pandas import DataFrame
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import tushare as ts
from WindPy import w
w.start();


import fdm

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2017 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [ ]:
pro = ts.pro_api()

In [2]:
client = MongoClient("192.168.56.1",27017)

In [3]:
tushare = fdm.Tushare(client)
db = tushare.daily_basic()

In [5]:
tushare.db.client

TypeError: 'MongoClient' object is not callable

In [ ]:
MSCI_CN = {'CNY': '302400L.MI',
           'USD': '302400.MI'}
data = w.wsd("302400L.MI", "close", "1999-12-20", "2019-08-26", "",usedf=True)[1]
data.sort_index(inplace = True)
data.to_json('msci_china.bz2')

In [ ]:
data = w.wsd("000300.SH", "close", "1999-12-20", "2019-08-26", "",usedf=True)[1]
data.sort_index(inplace = True)
data.to_json('hs300.bz2')

In [ ]:
index = pd.read_json('msci_china.bz2').sort_index().dropna()
index_ror = index.pct_change()

In [ ]:
index = pd.read_json('hs300.bz2').sort_index().dropna()
index_ror = index.pct_change()

In [ ]:
clean = fdm.CleanData(client)
cdb = clean.pricing()

tdb = fdm.TempDB(client)
actp = tdb['actPrice']

# Build active price vs CSI300
def actprice(code):
    start_date = datetime(2002,1,4)
    close = cdb.query({'code':code,'date':{'$gt':start_date}},['code','date','close'])
    if not close.empty:
        close.set_index(['date'],inplace = True)
        close = close.sort_index().copy()
        close['close'] = close['close'].pct_change()-index_ror['CLOSE']
        close.iat[0,1] = 0
        close['act_price'] = np.cumprod(close['close']+1)
        close.rename(columns={'close':'act_ror'},inplace=True)
        close.reset_index(inplace=True)
        return close
    else:
        return DataFrame

actp.drop()
codes = cdb.stock_codes()
for code in codes:
    df = actprice(code)
    actp.insert_many(df)



In [174]:
actp.create_indexs(['date','code'])

0

In [178]:
tdb = fdm.TempDB(client)
actp = tdb['actPrice']
mratio = tdb['mmRatio']

def maxmin_ratio(ts):
    maxv = np.max(ts)
    minv = np.min(ts)
    pt = ts[-1]
    return (minv-pt)/(minv-maxv)

#codes = actp.distinct('code')
mratio.drop()
for code in codes:
    data = actp.query({'code':code})
    data.set_index('date',inplace=True)
    data.sort_index(inplace=True)

    periods = [3,6,9,12,24,36]
    for n in periods:
        data['mm_ratio_'+str(n)] = data['act_price'].sort_index(ascending = False).rolling(20*n).apply(maxmin_ratio, raw=True)
    
    data.reset_index(inplace=True)
    mratio.insert_many(data)

4121 records inserted into tempDB.mmRatio
4073 records inserted into tempDB.mmRatio
13 records inserted into tempDB.mmRatio
3946 records inserted into tempDB.mmRatio
3837 records inserted into tempDB.mmRatio
4082 records inserted into tempDB.mmRatio
3578 records inserted into tempDB.mmRatio
3902 records inserted into tempDB.mmRatio
4075 records inserted into tempDB.mmRatio
3292 records inserted into tempDB.mmRatio
4072 records inserted into tempDB.mmRatio
4169 records inserted into tempDB.mmRatio
548 records inserted into tempDB.mmRatio
4136 records inserted into tempDB.mmRatio
4131 records inserted into tempDB.mmRatio
3163 records inserted into tempDB.mmRatio
4068 records inserted into tempDB.mmRatio
3903 records inserted into tempDB.mmRatio
3768 records inserted into tempDB.mmRatio
4181 records inserted into tempDB.mmRatio
3911 records inserted into tempDB.mmRatio
3138 records inserted into tempDB.mmRatio
4159 records inserted into tempDB.mmRatio
4183 records inserted into tempDB.mmR

In [179]:
mratio.create_indexs(['date','code'])

0

In [173]:
data = actp.query({'code':'000001.SZ'})
data.set_index('date',inplace=True)
data.sort_index(inplace=True)

In [ ]:
['CSI','SSE','SZSE','SW']
indexes = pro.index_basic(market='SW')
indexes.head()